In [5]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import pandas as pd
import re

In [6]:
# # connection into cloud db (central)
def cloud_connection():
    url = URL.create(
    "postgresql+psycopg2",
    username="irfan_admin",
    password="TradeAPP_IA_2026@",
    host="95.216.148.216",
    port=5432,
    database="trade_app",
)

    engine = create_engine(url, pool_pre_ping=True)

    with engine.connect() as conn:
        print('========|',conn.execute(text("SELECT current_user, current_database()")).fetchone(),'|========')

    print('*** ✅ SUCCESSFUL CLOUD CONNECTION ⛓️ ***')
    
    return engine

engine = cloud_connection()

========| ('irfan_admin', 'trade_app') |========
*** ✅ SUCCESSFUL CLOUD CONNECTION ⛓️ ***


In [8]:
# needs to read correct column names from cloud db
def read_sql_case_safe(engine, query: str):

    q = query.strip().rstrip(";")
    m = re.match(r'(?is)^\s*select\s+(?P<select>.+?)\s+from\s+(?P<table>[A-Za-z0-9_]+)\s*(?P<rest>.*)$', q)
    if not m:
        # tanıyamazsa aynen çalıştır
        return pd.read_sql_query(query, engine)

    select_part = m.group("select").strip()
    table = m.group("table").strip()
    rest = m.group("rest") or ""

    # tablonun gerçek kolon adlarını çek
    cols = pd.read_sql_query(text("""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema='public' AND table_name=:t
        ORDER BY ordinal_position
    """), engine, params={"t": table})["column_name"].tolist()

    # kolonu case-insensitive eşle
    col_map = {c.lower(): c for c in cols}

    if select_part == "*":
        fixed_select = "*"
    else:
        raw_cols = [c.strip() for c in select_part.split(",")]
        fixed_cols = []
        for rc in raw_cols:
            # fonksiyon/alias gibi şeyler varsa dokunmayalım
            if re.search(r'\(|\)|\s+as\s+|\s', rc, flags=re.I):
                fixed_cols.append(rc)
                continue

            real = col_map.get(rc.lower())
            if real:
                fixed_cols.append(f'"{real}"')
            else:
                fixed_cols.append(rc)
        fixed_select = ", ".join(fixed_cols)

    fixed_query = f'SELECT {fixed_select} FROM "{table}" {rest}'.strip()
    return pd.read_sql_query(fixed_query, engine)

In [14]:
import re
from sqlalchemy import text

def fn_read_data_cloud(schema, table_name, attributes=None):
    # table name validation
    if not re.fullmatch(r"[A-Za-z0-9_]+", table_name):
        raise ValueError("Invalid table name")
    if not re.fullmatch(r"[A-Za-z0-9_]+", schema):
        raise ValueError("Invalid schema name")
        
    # select cols
    if attributes is None:
        cols = '*'
    else:
        # kolon validation + quoting
        safe_cols = []
        for col in attributes:
            if not re.fullmatch(r"[A-Za-z0-9_]+", col):
                raise ValueError(f"Invalid column name: {col}")
            safe_cols.append(f'"{col}"')
        cols = ', '.join(safe_cols)
        
    # schema ve tablo ayrı ayrı tırnaklanmalı
    query = f'SELECT {cols} FROM "{schema}"."{table_name}"'
    
    return read_sql_case_safe(engine, query)

In [10]:
def fn_write_cloud(df, schema, name, if_exists,is_only_distinct = False,dist_col=None):
    #add/create direct
    if is_only_distinct == False:
        df.to_sql(name=name, con=engine,schema=schema, if_exists=if_exists, index=False)
    
    # check dist col, if dist, then append
    elif is_only_distinct == True:
        lst_exist_rows = list(fn_read_data_cloud(table_name=name)[dist_col])
        if dist_col not in lst_exist_rows:
            df_dist = df[~df[dist_col].isin(lst_exist_rows)]
            df_dist.to_sql(name=name, con=engine,schema=schema, if_exists='append', index=False)
            print(f'distinct data count {len(df_dist)} | COL: {name}')

In [11]:
def create_schema(schema_name, engine=engine):

    if not re.fullmatch(r"[A-Za-z0-9_]+", schema_name):
        raise ValueError("Invalid schema name")

    schema_name_clean = schema_name.lower()

    with engine.begin() as conn:

        # case-insensitive kontrol
        exists = conn.execute(
            text("""
                SELECT schema_name
                FROM information_schema.schemata
                WHERE lower(schema_name) = lower(:schema_name)
            """),
            {"schema_name": schema_name_clean}
        ).scalar()

        if exists:
            raise ValueError(f"Schema '{exists}' already exists")

        # her zaman lower-case oluştur (PostgreSQL best practice)
        conn.execute(
            text(f'CREATE SCHEMA "{schema_name_clean}"'))

In [16]:
fn_read_data_cloud('source','BIAS')

,TICKER,INTERVAL,RANGE,START_DATETIME,HIGH_DATETIME,HIGH_VALUE,HIGH_ROW_ID,END_DATETIME,ROW_COUNT_AFTER_HIGH,FRVP_JSON,...,PRICE_BIN,VAL_PERC,ROW_ID_FRVP,RUNTIME,FRVP_VWAP,FRVP_SKEW,FRVP_PEAKS,FRVP_STATE,TREND_BIAS,TREND_SCORE
0,ULKER.IS,daily,1mounth,2026-01-06 00:00:00.000000,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-05 00:00:00.000000,2,"{""134.0"": 1799910.942857143, ""135.0"": 1799910....",...,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_1mounth,None,135.973825,0.752079,0,BALANCED,0,0.251247
1,ULKER.IS,daily,3mounth,2025-11-07 00:00:00.000000,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-05 00:00:00.000000,2,"{""134.0"": 1799910.942857143, ""135.0"": 1799910....",...,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_3mounth,None,135.973825,0.752079,0,BALANCED,0,0.251247
2,ULKER.IS,daily,6mounth,2025-08-09 00:00:00.000000,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-05 00:00:00.000000,2,"{""134.0"": 1799910.942857143, ""135.0"": 1799910....",...,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_6mounth,None,135.973825,0.752079,0,BALANCED,0,0.251247
3,ULKER.IS,daily,1year,2025-02-05 00:00:00.000000,2026-02-04 00:00:00+03:00,139.500000,ID_ULKER.IS_20260204_0000_daily,2026-02-05 00:00:00.000000,2,"{""134.0"": 1799910.942857143, ""135.0"": 1799910....",...,1.0,0.68,ID_ULKER.IS_20260204_0000_daily_1year,None,135.973825,0.752079,0,BALANCED,0,0.251247
4,DOAS.IS,daily,1mounth,2026-01-06 00:00:00.000000,2026-01-26 00:00:00+03:00,237.600006,ID_DOAS.IS_20260126_0000_daily,2026-02-05 00:00:00.000000,9,"{""230.0"": 2193145.155641581, ""231.0"": 1165884....",...,1.0,0.68,ID_DOAS.IS_20260126_0000_daily_1mounth,None,227.458349,0.554833,1,BALANCED,0,0.199566
5,DOAS.IS,daily,3mounth,2025-11-07 00:00:00.000000,2026-01-26 00:00:00+03:00,237.600006,ID_DOAS.IS_20260126_0000_daily,2026-02-05 00:00:00.000000,9,"{""230.0"": 2193145.155641581, ""231.0"": 1165884....",...,1.0,0.68,ID_DOAS.IS_20260126_0000_daily_3mounth,None,227.458349,0.554833,1,BALANCED,0,0.199566
6,DOAS.IS,daily,6mounth,2025-08-09 00:00:00.000000,2026-01-26 00:00:00+03:00,237.600006,ID_DOAS.IS_20260126_0000_daily,2026-02-05 00:00:00.000000,9,"{""230.0"": 2193145.155641581, ""231.0"": 1165884....",...,1.0,0.68,ID_DOAS.IS_20260126_0000_daily_6mounth,None,227.458349,0.554833,1,BALANCED,0,0.199566
7,DOAS.IS,daily,1year,2025-02-05 00:00:00.000000,2026-01-26 00:00:00+03:00,237.600006,ID_DOAS.IS_20260126_0000_daily,2026-02-05 00:00:00.000000,9,"{""230.0"": 2193145.155641581, ""231.0"": 1165884....",...,1.0,0.68,ID_DOAS.IS_20260126_0000_daily_1year,None,227.458349,0.554833,1,BALANCED,0,0.199566
8,DOAS.IS,daily,2year,2024-02-06 00:00:00.000000,2024-04-17 00:00:00+03:00,313.906046,ID_DOAS.IS_20240417_0000_daily,2026-02-05 00:00:00.000000,453,"{""296.0"": 1038600.0011695907, ""297.0"": 1038600...",...,1.0,0.68,ID_DOAS.IS_20240417_0000_daily_2year,None,200.962133,0.213475,26,BALANCED,0,0.128085
9,CIMSA.IS,daily,1mounth,2026-01-06 00:00:00.000000,2026-01-28 00:00:00+03:00,53.950001,ID_CIMSA.IS_20260128_0000_daily,2026-02-05 00:00:00.000000,7,"{""50.0"": 18444991.066666666, ""51.0"": 33416658....",...,1.0,0.68,ID_CIMSA.IS_20260128_0000_daily_1mounth,None,51.915490,0.588109,1,BALANCED,0,0.152865
